In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import os
import sys 
import configparser
import pandas as pd 
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm

from matplotlib import cycler

from os.path import join as oj
import pickle as pkl
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression,ElasticNet,Lasso, Ridge
from sklearn.model_selection import cross_val_score
sys.path.insert(0, "../src")

import severson_data

import seaborn as sns
import pickle as pkl
pd.set_option('display.float_format', lambda x: '%.2f' % x)

config = configparser.ConfigParser()
config.read('../config.ini');
result_path =config['PATHS']['result_path'] 

# load data

In [4]:
if 'data_dict' not in locals(): # just takes a lot of time
    data_path = config['DATASET']['severson_path'] 
    bat_dicts = severson_data.load_data_single(data_path)
data_dict = {**bat_dicts[0], **bat_dicts[1], }

x, y, c, var  = severson_data.get_capacity_input(data_dict, num_offset=0, start_cycle = 10, stop_cycle = 100)

x = severson_data.preprocess_x(x,y)


C:\Users\lauri\AppData\Roaming\Python\Python39\site-packages\scipy\interpolate\interpolate.py:605: RuntimeWarning: invalid value encountered in true_divide
  slope = (y_hi - y_lo) / (x_hi - x_lo)[:, None]


In [5]:
train_idxs , val_idxs,test_idxs= severson_data.get_split(len(x), seed =42)




augmented_data =np.hstack([c, var,]) 
 
qc_variance_scaler = StandardScaler().fit(augmented_data[train_idxs]) 
augmented_data = qc_variance_scaler.transform(augmented_data)

x_aged, y_aged, c_aged, var_aged  = severson_data.get_capacity_input(bat_dicts[2] , num_offset=0, start_cycle = 10, stop_cycle = 100)

augmented_data_aged =np.hstack([c_aged, var_aged, ]) 

augmented_data_aged = qc_variance_scaler.transform(augmented_data_aged)

# fit and eval model

In [6]:

max_score = 0
max_alpha = 0
for alpha in range(-10,10):


        reg = Ridge(alpha = np.exp(alpha))
        cur_score = cross_val_score(reg, augmented_data[train_idxs], np.log(y[train_idxs]), cv=4).mean()
        if cur_score > max_score:
            max_score = cur_score
            max_alpha = np.exp(alpha)
reg = Ridge(alpha = max_alpha).fit(augmented_data[train_idxs], np.log(y[train_idxs]))
print(reg.score(augmented_data[val_idxs], np.log(y[val_idxs])))

0.7224228978688421


In [25]:

max_score = 0
max_alpha = 0
max_l1 =0
for alpha in range(-10,10):
    for l1_ratio in range(1,10, 1):

        reg = ElasticNet(alpha = np.exp(alpha),l1_ratio=l1_ratio/10, )
        cur_score = cross_val_score(reg, augmented_data[train_idxs], np.log(y[train_idxs]), cv=4).mean()
        if cur_score > max_score:
            max_score = cur_score
            max_alpha = np.exp(alpha)
            max_l1 = l1_ratio/10

            


In [26]:
reg = ElasticNet(alpha = max_alpha, l1_ratio=max_l1).fit(augmented_data[train_idxs], np.log(y[train_idxs]))
print(reg.score(augmented_data[val_idxs], np.log(y[val_idxs])))

0.7401740873231863


In [27]:
reg = ElasticNet(alpha = max_alpha,l1_ratio=max_l1 ).fit(augmented_data[train_idxs], np.log(y[train_idxs]))


In [28]:
for i in [6,10,20, 30, 40]:
    reg = ElasticNet(alpha = max_alpha,l1_ratio=max_l1 ).fit(augmented_data[train_idxs[:i]], np.log(y[train_idxs[:i]]))

    rmse_test = np.sqrt(np.square(np.exp(reg.predict(augmented_data[test_idxs])) - y[test_idxs]).mean())
    print("RMSE Test: {}".format(np.round(rmse_test)))

RMSE Test: 136.0
RMSE Test: 108.0
RMSE Test: 144.0
RMSE Test: 137.0
RMSE Test: 152.0


In [29]:
rmse_val = np.sqrt(np.square(np.exp(reg.predict(augmented_data[val_idxs])) - y[val_idxs]).mean())
rmse_train = np.sqrt(np.square(np.exp(reg.predict(augmented_data[train_idxs])) - y[train_idxs]).mean())
print("RMSE Train: {}".format(np.round(rmse_train)))
print("RMSE Val: {}".format(np.round(rmse_val)))


RMSE Train: 120.0
RMSE Val: 145.0


In [30]:
rmse_aged = np.sqrt(np.square(np.exp(reg.predict(augmented_data_aged)) - y_aged).mean())
print("RMSE Aged: {}".format(np.round(rmse_aged)))
rmse_test = np.sqrt(np.square(np.exp(reg.predict(augmented_data[test_idxs])) - y[test_idxs]).mean())
print("RMSE Test: {}".format(np.round(rmse_test)))

RMSE Aged: 204.0
RMSE Test: 152.0


# fold ensemble